Important steps to do before running the code blocks below!! 🚀🚀
1. Create a data folder and upload your csv data into.
2. Create a pieline folder to store metadata from the various components

In [ ]:
!pip install tfx

In [ ]:
!pip install dask[complete]

In [1]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tensorflow as tf
import os
import pprint
import dask.dataframe as dd

pp = pprint.PrettyPrinter()


In [2]:
context = InteractiveContext(pipeline_root='pipeline')

# 1. Data Cleaning with Dask 

In [8]:
df1 = dd.read_csv('data/*.csv')
df1.head()

,username,rating,helpful,total,date,title,review
0,Imme-van-Gorp,7,102,123,30 January 2019,Unfortunately the ending ruined an otherwise ...,This movie is full of suspense. It makes you g...
1,sonofocelot-1,5,385,500,10 May 2016,...oh dear Abrams. Again.\n,I'll leave this review fairly concise. <br/><b...
2,mhodaee,5,110,143,4 August 2017,"Fantastic, gripping, thoroughly enjoyable, un...",I give the 5/10 out of the credit I owe to the...
3,fil-nik09,5,73,100,5 October 2016,Hmmm...\n,"First of all, I must say that I was expecting ..."
4,DVR_Brale,7,42,56,27 July 2016,Slow building & plot alternating claustrophob...,I've always loved movies with strong atmospher...


In [9]:
#dropping unnecessary columns from the dataset
df2 = df1[['review', 'rating']]
df2.head()

,review,rating
0,This movie is full of suspense. It makes you g...,7
1,I'll leave this review fairly concise. <br/><b...,5
2,I give the 5/10 out of the credit I owe to the...,5
3,"First of all, I must say that I was expecting ...",5
4,I've always loved movies with strong atmospher...,7


In [10]:
#drop all rows with the null rating
df3 = df2[df2.rating != 'Null']
df3.head()

,review,rating
0,This movie is full of suspense. It makes you g...,7
1,I'll leave this review fairly concise. <br/><b...,5
2,I give the 5/10 out of the credit I owe to the...,5
3,"First of all, I must say that I was expecting ...",5
4,I've always loved movies with strong atmospher...,7


In [11]:
#Change all ratings to sentiment types
#ratings are initially strings so we need to change them to integers

df3.rating = df3.rating.astype('int')
df3.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 2 entries, review to rating
dtypes: object(1), int64(1)

In [12]:
df3['sentiment'] = (df3.rating > 5).astype('int')
df3.head()

,review,rating,sentiment
0,This movie is full of suspense. It makes you g...,7,1
1,I'll leave this review fairly concise. <br/><b...,5,0
2,I give the 5/10 out of the credit I owe to the...,5,0
3,"First of all, I must say that I was expecting ...",5,0
4,I've always loved movies with strong atmospher...,7,1


In [13]:
#convert dataframe to csv
df3[['review', 'sentiment']].to_csv('./data_in.csv', index=False)

['/content/./data_in.csv/00.part',
 '/content/./data_in.csv/01.part',
 '/content/./data_in.csv/02.part',
 '/content/./data_in.csv/03.part',
 '/content/./data_in.csv/04.part',
 '/content/./data_in.csv/05.part',
 '/content/./data_in.csv/06.part',
 '/content/./data_in.csv/07.part',
 '/content/./data_in.csv/08.part',
 '/content/./data_in.csv/09.part',
 '/content/./data_in.csv/10.part',
 '/content/./data_in.csv/11.part',
 '/content/./data_in.csv/12.part',
 '/content/./data_in.csv/13.part',
 '/content/./data_in.csv/14.part',
 '/content/./data_in.csv/15.part',
 '/content/./data_in.csv/16.part',
 '/content/./data_in.csv/17.part',
 '/content/./data_in.csv/18.part',
 '/content/./data_in.csv/19.part',
 '/content/./data_in.csv/20.part',
 '/content/./data_in.csv/21.part',
 '/content/./data_in.csv/22.part',
 '/content/./data_in.csv/23.part',
 '/content/./data_in.csv/24.part',
 '/content/./data_in.csv/25.part',
 '/content/./data_in.csv/26.part',
 '/content/./data_in.csv/27.part',
 '/content/./data_in

# 2. Pipeline

## 2.1 Data Ingestion

## 2.1.1 ExampleGen

In [14]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
        ])
)

In [ ]:
#!mv data/sample_data/ ./

In [15]:
example_gen = CsvExampleGen(input_base='data_in.csv', output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:60,total_bytes:60603323,xor_checksum:2,sum_checksum:98475805066"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
train_url = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')
#eval_url = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-eval')

In [17]:
tfrecord_filenames = [os.path.join(train_url, name) for name in os.listdir(train_url)]
#ev_tfrecord_filenames = [os.path.join(eval_url, name) for name in os.listdir(eval_url)]

In [18]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
#ds_eval = tf.data.TFRecordDataset(ev_tfrecord_filenames, compression_type='GZIP')

In [19]:
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)


features {
  feature {
    key: "review"
    value {
      bytes_list {
        value: "Anyone who lives in the world and follows movies has a pretty good idea of the main concept behind a quiet place: there are beings that will kill you if you make a noise.\nThe film does very little to try to explain where these beings come from, all we know is how long they\'ve been there for and that they have change the face of the planet in a pretty radical way.<br/><br/>We follow the Abbot family, who lives in a remote country house with an elaborate system to keep each other safe, but the main thing is that they have become very skilled at being very quiet.<br/><br/>The incredible result of that premise is that the film has very little dialogue and instead, makes great use of visuals and sound. And there are truly stunning set pieces in this film, and without spoiling anything, Emily Blunt gives a stellar performance, as usual. <br/><br/>Frustratingly, because the film chooses to concentrate on

In [ ]:
#for tfrecord in ds_eval.take(2):
#  ev_serialized_example = tfrecord.numpy()
#  ev_example = tf.train.Example()
#  ev_example.ParseFromString(ev_serialized_example)
#  pp.pprint(ev_example)

# 2.2 Data Validation

## 2.2.1 StatisticsGen

In [20]:
from tfx.components import StatisticsGen


In [21]:
statistics_gen = StatisticsGen(
    examples= example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [22]:
context.show(statistics_gen.outputs['statistics'])

## 2.2.2 SchemaGen

In [23]:
from tfx.components import SchemaGen

In [25]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

In [26]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [27]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',INT,required,,-


## 2.2.3 ExampleValidator

In [29]:
from tfx.components import ExampleValidator

In [30]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = schema_gen.outputs['schema']
)

In [31]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "pipeline/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [32]:
context.show(example_validator.outputs['anomalies'])

# 2.3 Data Preprocessing